# Howamidoing Demo

How Am I Doing? A tool to calculate your final letter grade of a course that takes into the effect of curved grading policies, assignment drops, clobber policies, etc.

The hierarchy of the objects in this library can be viewed as:

- `Course`: the course, e.g. **MATH 100**.
  - `CurvedSingleAssignment`: an assignment that is graded on curved basis, e.g. **Midterm 1**.
  - `UncurvedSingleAssignment`: an assignment that is graded on absolute scale, e.g. **Class Participation**.
  - `CurvedAssignmentGroup`: a group of assignment that is all graded on a curved basis and is aggregated when calculating the score towards the final grade. E.g., **Projects**.
    - `Assignment`: **Project 1**.
    - ...
  - `UncurvedAssignmentGroup`: a group of assignment that is all graded on the absolute scale and is aggregated when calculating the score towards the final grade. E.g., **Homeworks**.
    - `Assignment`: **Homework 1**.
    - ...
  - ...

# Comprehensive Example

## Course Policy

| Component | Weight |
| --------- | ------ |
| Participation | 5% |
| Homeworks | 15% |
| Projects | 20% |
| Midterm 1 | 15% |
| Midterm 2 | 15% |
| Final | 30% |

- Participation: Uncurved.
- Homeworks: 10 in total, all uncurved. Allow dropping 2 lowest score.
- Projects: 4 in total, all curved. Allow dropping 1 lowest score (based on zscore).
- Midterm 1: Curved.
- Midterm 2: Curved.
- Final: Curved.

**Clobber Policy**: Your grade on the final can be clobbered for either midterms, but not both. 

## Your Scores

- Participation: 100/100
- Homeworks: [20/20, 19/20, 28/30, 23/25, 12/15, 20/20, 20/20, 0/30, 30/30, 19/21]
- Projects:
  - Project 1: 81/100. Class mean = 76, class standard deviation = 12.4
  - Project 2: 68/100. Class mean = 70, class standard deviation = 23.2
  - Project 3: 94/100. Class mean = 90, class standard deviation = 11.2
  - Project 1: 85/100. Class mean = 91, class standard deviation = 14.8
- Midterm 1: 79/130. Class mean = 84.3, class standard deviation = 24.6
- Midterm 2: 118/187. Class mean = 106, class standard deviation = 28.3
- Final: 172/300. Class mean = 174, class standard deviation = 26.5

Here is the example of how to calculate this complicated grades in `howamidoing`.

In [2]:
from howamidoing import *

math100 = Course(name="MATH100")
math100.add_uncurved_single(0.05, "Participation")
hws = math100.add_uncurved_group